In [1]:
import numpy as np 
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt 
import time 

#### Paths used

In [2]:
path = '/home/marcos/Documentos/6.-Post_Doctorado_CIDE_Deep_Learning/Proyectos/Monitor_Vientos/2.-Viento_Medio/Preprocessing_Data/'
path_aemet = '3.-data_with_NAN_values_corrected/'
nc_mask_aemet = 'mask_aemet_daily-wind_1961-2021_010x010_144.nc'
path_out = '2.-Mask_creation/Mask_with_HoldOut/Stations_removed_randomly_percentage/'

#### Definition of the function 

In [3]:
def random_drop(ds, ratio):

    #output dataframe
    df = None

    #I save the dates
    dates = pd.date_range(ds.time[0].values, ds.time[-1].values, freq="D")
    #dates = ds.time
    
    #Now, it expresses the deactivated ratio of AEMET stations
    ratio_value = 1 - ratio

    for dd in dates:
        #ds.isel(time=-126)["mask"].plot()

        #Take just 1 time and forget the time dimension. I only want to deactivate stations in the XY dimension
        coords_mask = ds.sel(time=dd).to_dataframe().drop(columns ="time")#reset_index()

        daout = coords_mask.where(coords_mask.isnull(), 0)

        aemet_stations = coords_mask.loc[coords_mask["mask"]>0]

        shuffle_index = np.random.permutation(len(aemet_stations))

        est_index_down = shuffle_index[:int(len(aemet_stations) * ratio_value)]

        mod_ae_stat = aemet_stations.copy()
        mod_ae_stat["mask"] = 1
        mod_ae_stat["mask"].iloc[est_index_down] = 0

        mod_ae_stat = mod_ae_stat.loc[mod_ae_stat["mask"] == 1]

        if len(mod_ae_stat)>0:
            daout.loc[mod_ae_stat.index] = 1
            daout.columns = [dd]
            
            if df is None:
                df = daout 
            else:
                df = df.join(daout)

            #This only works if I use pd_range for dates.
            #print()
            if (int(dd.strftime("%d")) * int(dd.strftime("%m"))) % 300 == 0:
                print(dd.strftime("%Y"))
            #print(dd.strftime("%Y-%m-%d"))
            #print("AEMET station size: ", aemet_stations.size, "mod_ae_stat size: ", mod_ae_stat.size, "daout size: ", daout.size)
            #print("% to be completed: ", df.shape[1]/len(dates)*100, "%")
                            
            #This only works if I use ds.time for dates.
            #print(dd, aemet_stations.size, mod_ae_stat.size, daout.size, df.shape[1]/len(dates)*100, "%")

    df = df.stack()
    df.index.names = ["latitude", "longitude", "time"]
    df.name = "mask"

    dsmask = df.to_xarray()

    return dsmask

#### Main function

In [4]:
if __name__ == "__main__":

    dsmask = xr.open_dataset(path + path_aemet + nc_mask_aemet)

    dsmask = dsmask.sel(time=slice("2010-01-01", "2021-12-31"))

    np.random.seed(42)

    ratios = [0.3, 0.7]
    ratios = [0.3]

    for ratio in ratios:
        t1 = time.time()
        ds1 = random_drop(dsmask, ratio)
        ds1 = ds1.transpose("time", "latitude", "longitude")

        current_indexes = ds1.indexes
        desired_order = ['time', 'latitude', 'longitude']
        reordered_indexes = {index_name: current_indexes[index_name] for index_name in desired_order}
        ds1 = ds1.reindex(reordered_indexes)
        #ds1 = ds1.to_dataset(name='mask')

        t2 = time.time()
        #print("ra", t2)
        
        fout = "mask_aemet_daily-wind_2010-2021_144_" + str(ratio).replace(".", "p") + "percentage-stations.nc"

        ds1.to_netcdf(path + path_out + fout)

        t3 = time.time()

        print("ratio: ", ratio)
        
        print()
        print("t1: ", t1)
        print("t2: ", t2)
        print("t3: ", t3)

        print()
        print("D_t_21: ", t2 - t1)
        print("D_t_32: ", t3 - t2)
